# Preparation
- Import libraries
- Import Data

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
import keras

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import os

import math


from tensorflow.keras.utils import Sequence
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import datetime as dt

import warnings
warnings.filterwarnings('ignore')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)  # turn off deprecation warnings



2023-03-28 19:33:29.635448: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-28 19:33:29.698410: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-28 19:33:29.699928: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 19:33:30.682933: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('../data/final_dataset22.csv')
df

,datetime,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,...,NI,RICE,XAU,IRD,SUGAR,ETHANOL,CORN,COTTON,TIN,ZNC
0,1641017460000,47249.44,47281.68,47249.44,47271.88,10.37408,1641017519999,4.903711e+05,403,6.99871,...,0.62547,14.632537,0.000547,4147.926037,0.189924,2.136423,5.928399,1.129187,1.222731,0.102048
1,1641017520000,47271.87,47290.00,47271.87,47289.97,4.11304,1641017579999,1.944688e+05,343,2.50207,...,0.62547,14.632537,0.000547,4147.926037,0.189924,2.136423,5.928399,1.129187,1.222731,0.102048
2,1641017580000,47289.96,47320.00,47287.60,47300.72,13.45235,1641017639999,6.363889e+05,472,6.83725,...,0.62547,14.632537,0.000547,4147.926037,0.189924,2.136423,5.928399,1.129187,1.222731,0.102048
3,1641017640000,47300.71,47324.42,47293.01,47307.04,6.17338,1641017699999,2.920746e+05,416,3.06969,...,0.62547,14.632537,0.000547,4147.926037,0.189924,2.136423,5.928399,1.129187,1.222731,0.102048
4,1641017700000,47307.03,47307.04,47258.73,47259.14,4.41853,1641017759999,2.088964e+05,415,1.66964,...,0.62547,14.632537,0.000547,4147.926037,0.189924,2.136423,5.928399,1.129187,1.222731,0.102048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525525,1672548960000,16521.42,16521.93,16516.49,16518.76,83.64312,1672549019999,1.381688e+06,2545,41.92976,...,29975.00000,18.000000,0.000548,4800.000000,0.200500,2.161000,6.785000,0.830000,19391.166190,3012.500000
525526,1672549020000,16518.76,16519.22,16517.03,16517.41,48.86392,1672549079999,8.071365e+05,1783,21.38648,...,29975.00000,18.000000,0.000548,4800.000000,0.200500,2.161000,6.785000,0.830000,19391.166190,3012.500000
525527,1672549080000,16517.39,16520.64,16517.28,16519.37,61.07865,1672549139999,1.008950e+06,2350,36.47444,...,29975.00000,18.000000,0.000548,4800.000000,0.200500,2.161000,6.785000,0.830000,19391.166190,3012.500000
525528,1672549140000,16519.37,16522.81,16511.92,16521.85,173.39683,1672549199999,2.864058e+06,4050,89.59732,...,29975.00000,18.000000,0.000548,4800.000000,0.200500,2.161000,6.785000,0.830000,19391.166190,3012.500000


In [3]:
#df.drop(columns = ['Unnamed: 0'], inplace=True)
#df.to_csv('../data/final_dataset22.csv', index = False)

# Defining Normalization Technique for the Process

### Analytical Approach
- We will iterate using the 7 scalers/normalizers on Scikit-Learn Library over a defined dataset (BTCUSD - 2022 - Minute) across different models and see after being trained with each of the different models the differences on the performance.
- Each model will be trained over 30 epochs
- We will measure the performance by using two simple permormance metrics: _MSE_ and _MAE_ 


# Timeseries Parameters

In [4]:


# delay: time in future that will be predicted
delay = 2 * 60


# sequence length: sequence lenght of each sample
sequence_length = delay * 12


# sequence_stride: period between sequences
# First sequence starts at t0
# Second sequence will start at t1 with sequence_stride=1 or at t5 with sequence_stride=5
sequence_stride = 1


# sampling rate: period between timesteps within the sequence
# Sequence with rate=1 : t1,t2...tn
# Sequence with rate=3 : t1,t3...tn*3
sampling_rate = 1

#batch_size: Number of timeseries samples in each batch (except maybe the last one). 
#If None, the data will not be batched (the dataset will yield individual samples).
# Huge impact in performance.
# Tip, should be multiple of 8
batch_size = 64

# Understanding our parameters
msg = f"The timeseries will consist of batches containing {batch_size} sequences of {sequence_length} samples."

msg += f"\nFinally our target is {delay} timesteps in the future"
print(msg)

The timeseries will consist of batches containing 64 sequences of 1440 samples.
Finally our target is 120 timesteps in the future


In [5]:
# Convert the DataFrame to a NumPy array
raw_data = df.values
close = df['close'].values
close = close.reshape((len(close),1))
# Display the NumPy array
print(type(raw_data))
close.shape

<class 'numpy.ndarray'>


(525530, 1)

In [6]:
num_train_samples = int(0.5 * len(raw_data))  # Train
num_val_samples = int(0.25 * len(raw_data))   # Validation
num_test_samples = len(raw_data) - num_train_samples - num_val_samples  # Test
print("num_train_samples:", num_train_samples)
print("num_val_samples:", num_val_samples)
print("num_test_samples:", num_test_samples)

num_train_samples: 262765
num_val_samples: 131382
num_test_samples: 131383


In [7]:

train_dataset = keras.preprocessing.timeseries_dataset_from_array(
                    raw_data[:-delay],
                    targets=close[delay:],
                    sampling_rate=sampling_rate,
                    sequence_stride=sequence_stride,
                    sequence_length=sequence_length,
                    shuffle=True, # Shouldn't the shuffle be set to 0?
                    seed=33,
                    batch_size=batch_size,
                    start_index=0,
                    end_index=num_train_samples)

print("Done Train")


val_dataset = keras.preprocessing.timeseries_dataset_from_array(
                    raw_data[:-delay],
                    targets=close[delay:],
                    sampling_rate=sampling_rate,
                    sequence_stride=sequence_stride,
                    sequence_length=sequence_length,
                    shuffle=True,
                    seed=33,
                    batch_size=batch_size,
                    start_index=num_train_samples,
                    end_index=num_train_samples + num_val_samples)

print("Done Validation")      
      
test_dataset = keras.preprocessing.timeseries_dataset_from_array(
                    raw_data[:-delay],
                    targets=close[delay:],
                    sampling_rate=sampling_rate,
                    sequence_stride=sequence_stride,
                    sequence_length=sequence_length,
                    shuffle=True,
                    seed=33,
                    batch_size=batch_size,
                    start_index=num_train_samples + num_val_samples)

print("Done Test")

Done Train
Done Validation
Done Test


## Build Models

In [8]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))



#history = model.fit(train_dataset,
#                    epochs=30,
#                    validation_data=val_dataset,
#                    callbacks=callbacks)

#### LSTM

In [9]:

x = layers.LSTM(32)(inputs)
outputs = layers.Dense(1)(x)
lstm = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("/models/lstm",
   save_best_only=True) 
]

lstm.compile(optimizer='rmsprop',
              loss=['mse', 'mae'],
              metrics=['accuracy', 'mse', 'mae'])

2023-03-28 19:33:41.435860: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-28 19:33:41.437385: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-28 19:33:41.438621: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

#### Simple RNN

In [10]:

x = layers.SimpleRNN(32)(inputs)
outputs = layers.Dense(1)(x)
simple_rnn = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("C:/models/simple_rnn",
   save_best_only=True)
]

simple_rnn.compile(optimizer='rmsprop',
              loss=['mse', 'mae'],
              metrics=['accuracy', 'mse', 'mae'])

#### Gated Recurrent Unit (GRU)

In [11]:

x = layers.GRU(64)(inputs)
outputs = layers.Dense(1)(x)
gru = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("C:/models/gru",
   save_best_only=True)
]

gru.compile(optimizer='rmsprop',
              loss=['mse', 'mae'],
              metrics=['accuracy', 'mse', 'mae'])



2023-03-28 19:33:41.739818: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-28 19:33:41.741320: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-28 19:33:41.742658: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

#### Bidirectional LSTM

In [12]:

x = layers.Bidirectional(layers.LSTM(16))(inputs)
outputs = layers.Dense(1)(x)
bi_lstm = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("C:/models/bidirectional_lstm",
   save_best_only=True)
]

bi_lstm.compile(optimizer='rmsprop',
              loss=['mse', 'mae'],
              metrics=['accuracy', 'mse', 'mae'])


2023-03-28 19:33:42.121143: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-28 19:33:42.122667: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-28 19:33:42.123954: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

## Normalizing Data

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer, PowerTransformer, Normalizer

# Define a list of scalers
scalers = [('standard', StandardScaler()), 
           ('minmax', MinMaxScaler()), 
           ('robust', RobustScaler()), 
           ('quantile', QuantileTransformer()), 
           ('power', PowerTransformer()), 
           ('normalize', Normalizer())]

# Loop through each scaler, apply the scaler to your dataset, and save the processed dataset to a CSV file
for scaler_name, scaler in scalers:
    # Apply the scaler to your dataset
    df_processed = pd.DataFrame(scaler.fit_transform(df.values), columns=df.columns)
    
    # Save the processed dataset to a CSV file
    prefix = '../data/normalized/'
    filename = prefix = scaler_name + '_processed_dataset.csv'
    df_processed.to_csv(filename, index=False)
    print(filename)

standard_processed_dataset.csv
